In [1]:
!pip3 install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip3 install -e git+https://github.com/openai/CLIP.git@main#egg=clip

import argparse, os, sys, datetime, glob, importlib, csv
import pytorch_lightning
from omegaconf import OmegaConf

import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import random_split
from torch.utils.data import DataLoader

from ldm.models.autoencoder import AutoencoderKL
from ldm.util import instantiate_from_config

dir_path = os.getcwd()
data_dir = os.path.join(dir_path, "datasets")


/Users/armanommid/anaconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'taming.modules.vqvae'

In [2]:
class StandardDataset(pytorch_lightning.LightningDataModule):
    def __init__(self, dataset_name, data_dir, batch_size=64, num_workers=0):
        super().__init__()
        self.dataset_name = dataset_name
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers

    def prepare_data(self):
        dataset = getattr(torchvision.datasets, self.dataset_name)(self.data_dir, train=True, download=True)

    def setup(self):
        dataset = getattr(torchvision.datasets, self.dataset_name)(self.data_dir, train=True, download=False, transform=T.ToTensor())
        total = len(dataset)
        n_train = int(total * 0.8)
        n_val = int(total - n_train)
        self.train_dataset, self.val_dataset = random_split(dataset, [n_train, n_val])

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )


dataset_name = "CIFAR10"
dataset = StandardDataset(dataset_name, data_dir)
dataset.prepare_data()
dataset.setup()


Files already downloaded and verified


In [3]:

config = "configs/autoencoder/autoencoder_kl_8x8x64.yaml"
config = OmegaConf.load(config)

model = instantiate_from_config(config.model)
model.learning_rate = config.model.base_learning_rate

trainer = pytorch_lightning.Trainer(max_epochs=3)

trainer.fit(model, dataset)


making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 64, 8, 8) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/armanommid/anaconda3/envs/ldm/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name            | Type                   | Params
-----------------------------------------------------------
0 | encoder         | Encoder                | 42.6 M
1 | decoder         | Decoder                | 61.2 M
2 | loss            | LPIPSWithDiscriminator | 17.5 M
3 | quant_conv      | Conv2d                 | 16.5 K
4 | post_quant_conv | Conv2d                 | 4.2 K 
-----------------------------------------------------------
106 M     Trainable params
14.7 M    Non-trainable params
121 M     Total params
485.448   Total estimated model params size (MB)


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/armanommid/anaconda3/envs/ldm/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/armanommid/anaconda3/envs/ldm/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/1250 [00:00<00:00, 1864.96it/s]  

/Users/armanommid/anaconda3/envs/ldm/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
